In [ ]:
%load_ext autoreload
%autoreload 2


# Traci

In [ ]:
path='/usr/share/sumo/tests/complex/tutorial/traci_tls'
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import random

# we need to import python modules from the $SUMO_HOME/tools directory
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import traci  # noqa

In [ ]:
# this script has been called from the command line. It will start sumo as a
# server, then connect and run
sumoBinary = checkBinary('sumo')


# this is the normal way of using traci. sumo is started as a
# subprocess and then the python script connects and runs
traci.start([sumoBinary, "-c", f"{path}/data/cross.sumocfg",
                         "--tripinfo-output", "tripinfo.xml"])

## Run a simulation until all vehicles have arrived

In [ ]:
while traci.simulation.getMinExpectedNumber() > 0:
  traci.simulationStep()
traci.close()

## TrafficLights

In [ ]:
traci.simulationStep()
if traci.trafficlight.getPhase("0") == 2:
    # we are not already switching
    if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
        # there is a vehicle from the north, switch
        traci.trafficlight.setPhase("0", 3)
    else:
        # otherwise try to keep green for EW
        traci.trafficlight.setPhase("0", 2)

## Vehicles

In [ ]:
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    if len(traci.vehicle.getIDList()) > 0:
        break

In [ ]:
traci.vehicle.getIDList()

In [ ]:
traci.vehicle.getSpeed("left_0")
traci.vehicle.getPosition("left_0")

# ATSC

## Large Grid
25 agents, 12 dim observation

In [ ]:
from algorithms.envs.NCS.large_grid_env import LargeGridEnv
import os
import configparser
import traci

In [ ]:
config_path = "algorithms/envs/NCS/config/config_ma2c_nc_grid.ini"
config_path = os.path.join(os.path.dirname("."), config_path)
config = configparser.ConfigParser()
config.read(config_path)
config = config['ENV_CONFIG']
env = LargeGridEnv(config)

In [ ]:
state = env.reset()

## Real Net
28 agents, observation space from 2 dim to 22 dim ...

In [ ]:
from algorithms.envs.NCS.real_net_env import RealNetEnv
import os
import configparser
import traci

In [ ]:
config_path = "algorithms/envs/NCS/config/config_ma2c_nc_net.ini"
config_path = os.path.join(os.path.dirname("."), config_path)
config = configparser.ConfigParser()
config.read(config_path)
config = config['ENV_CONFIG']
env = RealNetEnv(config)

In [ ]:
state = env.reset()

In [ ]:
import ray

In [ ]:
torch

In [ ]:
import torch

In [ ]:
x= torch.zeros((100, 100)).to(0)

In [ ]:
print(s)